In [ ]:
pip install PyPortfolioOpt

In [ ]:
pip install tslearn

In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

import warnings
warnings.filterwarnings('ignore')

In [76]:
# 데이터 병합
sectors_data = {}
excel_file_path = 'stocks_2000_2020_data_by_sector-2.xlsx'
snp_price_data = pd.ExcelFile(excel_file_path)
for sheet_name in snp_price_data.sheet_names:
    sheet_data = pd.read_excel(excel_file_path, sheet_name=sheet_name)
    sheet_data['Date'] = pd.to_datetime(sheet_data['Unnamed: 0'])
    sheet_data.set_index('Date', inplace=True)
    sheet_data.drop(columns='Unnamed: 0', inplace=True)
    sectors_data[sheet_name] = sheet_data

# 데이터프레임 생성
df = pd.DataFrame()
for sector, data in sectors_data.items():
    data.columns = [f"{sector}_{col}" for col in data.columns]
    if df.empty:
        df = data
    else:
        df = df.join(data, how='outer')

df


,Tele_VZ,Tele_DIS,Tele_T,Tele_CMCSA,Tele_EA,COND_AMZN,COND_HD,COND_MCD,COND_NKE,COND_SBUX,...,UTIL_NEE,UTIL_DUK,UTIL_D,UTIL_SO,UTIL_EXC,REES_AMT,REES_SPG,REES_CCI,REES_PLD,REES_PSA
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,53.903156,29.471687,35.498489,15.583333,25.265625,4.468750,65.187500,39.625000,6.015625,3.082031,...,5.156250,42.057114,19.062500,13.897975,12.080956,29.312500,20.999237,30.531250,19.812500,22.750000
2000-01-04,52.160721,31.198063,33.421452,14.416667,22.343750,4.096875,61.750000,38.812500,5.687500,2.984375,...,5.250000,42.710850,19.531250,14.050700,11.991797,29.000000,20.767200,30.500000,19.500000,22.750000
2000-01-05,53.903156,32.492844,33.940708,14.041667,22.328125,3.487500,63.000000,39.437500,6.015625,3.023438,...,5.421875,44.454151,20.062500,14.776144,12.571327,29.875000,20.941229,30.062500,19.250000,22.937500
2000-01-06,53.284874,31.198063,33.043808,14.583333,20.000000,3.278125,60.000000,38.875000,5.984375,3.132813,...,5.421875,45.652668,20.281250,14.814325,12.593616,29.562500,21.463308,29.000000,19.625000,23.812500
2000-01-07,52.891418,30.704813,33.327038,14.125000,20.593750,3.478125,63.500000,39.875000,5.984375,3.117188,...,5.578125,46.851189,20.281250,15.081594,12.593616,31.187500,22.391451,30.500000,20.000000,24.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-23,58.959999,173.550003,21.714502,50.040001,141.509995,159.263504,269.809998,212.020004,141.759995,102.059998,...,74.330002,89.040001,74.040001,59.380001,29.750357,217.399994,83.650002,152.460007,96.190002,227.000000
2020-12-24,58.840000,173.729996,21.669184,50.380001,141.809998,158.634506,270.920013,211.389999,141.600006,102.010002,...,74.980003,89.669998,74.230003,59.799999,29.835949,218.649994,84.370003,154.000000,97.239998,228.850006
2020-12-28,58.980000,178.860001,21.563444,51.110001,140.720001,164.197998,269.250000,214.020004,142.429993,104.339996,...,75.489998,90.250000,73.699997,59.990002,30.106991,221.070007,85.480003,156.250000,98.220001,229.139999


In [77]:
# 수익률 시계열 생성
return_df = np.log(df / df.shift(1))
return_df

,Tele_VZ,Tele_DIS,Tele_T,Tele_CMCSA,Tele_EA,COND_AMZN,COND_HD,COND_MCD,COND_NKE,COND_SBUX,...,UTIL_NEE,UTIL_DUK,UTIL_D,UTIL_SO,UTIL_EXC,REES_AMT,REES_SPG,REES_CCI,REES_PLD,REES_PSA
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,-0.032859,0.056926,-0.060292,-0.077817,-0.122898,-0.086884,-0.054174,-0.020718,-0.056089,-0.032198,...,0.018019,0.015424,0.024293,0.010929,-0.007407,-0.010718,-0.011111,-0.001024,-0.015899,0.000000
2000-01-05,0.032859,0.040664,0.015417,-0.026356,-0.000700,-0.161039,0.020041,0.015975,0.056089,0.013004,...,0.032214,0.040005,0.026837,0.050342,0.047196,0.029726,0.008345,-0.014448,-0.012903,0.008208
2000-01-06,-0.011537,-0.040664,-0.026781,0.037850,-0.110115,-0.061914,-0.048790,-0.014366,-0.005208,0.035537,...,0.000000,0.026604,0.010844,0.002581,0.001771,-0.010515,0.024625,-0.035983,0.019293,0.037438
2000-01-07,-0.007411,-0.015937,0.008535,-0.031933,0.029255,0.059222,0.056695,0.025398,0.000000,-0.005000,...,0.028411,0.025914,0.000000,0.017880,0.000000,0.053511,0.042334,0.050431,0.018928,0.013038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-23,-0.000509,0.018024,0.012249,0.000400,0.014808,-0.006649,-0.000852,0.000472,-0.004856,-0.003424,...,-0.005099,-0.005488,0.007320,-0.005710,-0.001198,-0.014113,-0.000836,-0.006733,-0.015064,-0.022606
2020-12-24,-0.002037,0.001037,-0.002089,0.006772,0.002118,-0.003957,0.004106,-0.002976,-0.001129,-0.000490,...,0.008707,0.007051,0.002563,0.007048,0.002873,0.005733,0.008570,0.010050,0.010857,0.008117
2020-12-28,0.002377,0.029101,-0.004892,0.014386,-0.007716,0.034470,-0.006183,0.012365,0.005844,0.022584,...,0.006779,0.006447,-0.007166,0.003172,0.009043,0.011007,0.013071,0.014505,0.010028,0.001266


In [78]:
# 월별 누적 로그 수익률 계산
cumulative_monthly_returns = return_df.resample('M').sum()
cumulative_monthly_returns

,Tele_VZ,Tele_DIS,Tele_T,Tele_CMCSA,Tele_EA,COND_AMZN,COND_HD,COND_MCD,COND_NKE,COND_SBUX,...,UTIL_NEE,UTIL_DUK,UTIL_D,UTIL_SO,UTIL_EXC,REES_AMT,REES_SPG,REES_CCI,REES_PLD,REES_PSA
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-31,0.032823,0.195140,-0.091859,-0.080711,-0.212073,-0.325207,-0.140817,-0.058458,-0.056089,0.260706,...,0.022473,0.179727,0.090829,0.121439,0.209022,0.202027,0.087242,0.035197,0.031058,-0.002751
2000-02-29,-0.235582,-0.065802,-0.127303,-0.075223,0.201504,0.064660,0.015334,-0.163109,-0.470003,0.093177,...,-0.088224,-0.174560,-0.129263,-0.146475,-0.112371,0.316869,-0.054630,0.019570,-0.012308,-0.027935
2000-03-31,0.222377,0.193291,0.109656,0.030772,-0.339853,-0.027601,0.114880,0.163109,0.331751,0.243574,...,0.176100,0.079249,0.046597,-0.019915,-0.011795,0.002535,-0.002677,0.160773,0.062989,-0.049357
2000-04-30,-0.018576,0.055979,0.039278,-0.056089,-0.162674,-0.193956,-0.132425,0.018228,0.091863,-0.393508,...,-0.017797,0.090972,0.157629,0.136759,0.122667,-0.058649,0.087235,0.013115,0.025826,0.063422
2000-05-31,-0.126414,-0.033506,-0.002857,-0.037555,0.054285,-0.133046,-0.142420,-0.060933,-0.013034,0.117381,...,0.093551,0.012959,0.016529,0.039317,0.052567,-0.226505,-0.037551,-0.382124,0.019636,-0.002797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-31,0.030666,0.120156,0.007745,0.045893,-0.015298,0.086601,0.071052,0.094430,0.136536,0.098714,...,-0.005466,-0.053320,-0.032485,-0.045518,-0.045029,-0.047959,0.084536,-0.041750,-0.034353,0.060754
2020-09-30,0.003705,-0.060890,-0.044589,0.031846,-0.067166,-0.091653,-0.026052,0.027576,0.115108,0.017020,...,-0.005784,0.097413,0.006227,0.038351,-0.031653,-0.030234,-0.047847,0.019713,-0.012248,0.047444
2020-10-31,-0.042932,-0.023072,-0.053678,-0.090910,-0.084609,-0.036409,-0.040415,-0.030015,-0.044466,0.012032,...,0.053589,0.039303,0.017706,0.057865,0.109296,-0.051265,-0.029338,-0.063858,-0.014213,0.028112


In [79]:
csv_file_path = "sp500_cumulative_returns.csv"
cumulative_monthly_returns.to_csv(csv_file_path)